In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sess = tf.InteractiveSession()
v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
tf.clip_by_value(v, 2.5, 4.5).eval() # 截断

array([[2.5, 2.5, 3. ],
       [4. , 4.5, 4.5]], dtype=float32)

In [3]:
v1 = tf.constant([[1.0, 2.0], [3.0, 4.0]])
v2 = tf.constant([[4.0, 6.0], [7.0, 8.0]])
print((v1*v2).eval()) # 矩阵逐元素相乘
print(tf.matmul(v1, v2).eval()) # 矩阵乘法

[[ 4. 12.]
 [21. 32.]]
[[18. 22.]
 [40. 50.]]


In [4]:
v = tf.constant([[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]])
tf.reduce_mean(v).eval()

3.5

In [7]:
# 自定义损失函数
v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])
print(tf.greater(v1, v2).eval()) # 逐元素比较
print(tf.where(tf.greater(v1, v2), v1, v2).eval()) # 如果第一个参数为True，则输出第二个参数，否则输出第三个参数(逐元素进行)

[False False  True  True]
[4. 3. 3. 4.]


In [1]:
# l1/l2 regularizer
import tensorflow as tf
weights = tf.constant([[1.0, -2.0], [-3.0, 4.0]])
with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(weights))) # l1正则化， 0.5为正则化比例
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(weights)))

5.0
7.5


In [ ]:
# 通过集合正则化多组不同的w
def get_weight(shape, lambda): # 在初始化参数时就将其加入正则化参数集合
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.l2_regularizer(lambda)(var))
    return var
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
batch_size = 8
layer_dimension = [2, 10, 10, 10, 1]
n_layers = len(layer_dimension)
current_layer = x # 当前层
in_dimension = layer_dimension[0] # 输入层节点数
for i in range(1, n_layers):
    out_dimension = layer_dimension[i] # 当前层输出维度
    weight = get_weight([in_dimension, out_dimension], 0.001) # 权重
    current_layer = tf.nn.relu(tf.matmul(current_layer, weight) + bias) # 当前层输出， 下层输入
    in_dimension = layer_dimension[i] # 下层输入维度
mse_loss = tf.reduce_mean(tf.square(y-current_layer))
tf.add_to_collection('losses', mse_loss) # 损失集合中已有正则化损失
loss = tf.add_n(tf.get_collection('losses')) # get_collection获取集合所有元素，相加得到最终损失